In [116]:
import os
import sys
import importlib

sys.path.append('../')

import utils
importlib.reload(utils)

import keras_core as keras
import jax
import jax.numpy as jnp
from jax import random, vmap, jit, grad
#assert jax.default_backend() == 'gpu'
import numpy as np
import pandas as pd
import time
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
#import elegy # pip install elegy. # Trying to do this with keras core instead.
from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions

import preprocess.preprocess
from preprocess.preprocess import INPUTS, PARAMETERS, PARAMETERS_SPECIFIED, RIGIDITY_VALS
from preprocess.preprocess import transform_input, untransform_input
from preprocess.preprocess import PARAMETERS_MIN, PARAMETERS_MAX
from chi2 import CalculateChi2

# Check new yearly data load

In [117]:
# Need version with correct path to the data
def index_mcmc_runs(file_version):
    """Make a list of combinations for which we want to run MCMC."""
    if file_version == '2023':
        experiments = ['AMS02_H-PRL2021', 'PAMELA_H-ApJ2013', 'PAMELA_H-ApJL2018']
        dfs = []
        for experiment_name in experiments:
            filename = f'../../data/2023/{experiment_name}_heliosphere.dat'
            df = utils.index_experiment_files(filename) 
            df['experiment_name'] = experiment_name
            df['filename_heliosphere'] = filename
            dfs.append(df)
        df = pd.concat(dfs, axis=0, ignore_index=0)

    elif file_version == '2024':
        filename = f'../../data/2024/yearly_heliosphere.dat'
        df = utils.read_experiment_summary(filename)
        df['experiment_name'] = 'yearly'
        df['filename_heliosphere'] = filename

    else: raise ValueError(f"Unknown file_version {file_version}. Must be '2023' or '2024'.")

    return df

In [118]:
SLURM_ARRAY_TASK_ID = 0
SLURM_ARRAY_JOB_ID = 0
DEBUG = True

# Version specifications
model_version = 'v3.0' # v2.0 is MSE NN, v3.0 is MAE NN
hmc_version = 'v25.0'
file_version = '2024'

# Select experiment parameters
df = index_mcmc_runs(file_version=file_version)  # List of all experiments (0-209) for '2023', 0-14 for '2024'
print(f'Found {df.shape[0]} combinations to run MCMC on. Performing MCMC on index {SLURM_ARRAY_TASK_ID}.')
df = df.iloc[SLURM_ARRAY_TASK_ID]

# Define parameters for HMC
seed = SLURM_ARRAY_TASK_ID + SLURM_ARRAY_JOB_ID
penalty = 1e6
integrate = False # If False, Chi2 is interpolated. If True, Chi2 is integrated.
par_equals_perr = False # If True, only 3 parameters will be sampled by the HMC and pwr1par==pwr1perr and pwr2par==pwr2perr
constant_vspoles = False # If True, vspoles is fixed to 400.0. If False, vspoles is specified in the data file.
specified_parameters = utils.get_parameters(df.filename_heliosphere, df.interval, constant_vspoles=constant_vspoles)

# Number of parameters for HMC to sample. 5 normally, 3 if par_equals_perr=True
if par_equals_perr:
    num_params = 3
else:
    num_params = 5

# Load observation data and define logprob. 
if file_version == '2023': 
    data_path = f'../../data/oct2022/{df.experiment_name}/{df.experiment_name}_{df.interval}.dat'  # This data is the same.
elif file_version == '2024': 
    year = 2000 + SLURM_ARRAY_TASK_ID # assumes only negative intervals. If otherwise, fix this
    data_path = f'../../data/2024/yearly/{year}.dat'
else:
    raise ValueError(f"Invalid file_version {file_version}. Must be '2023' or '2024'.")

model_path = f'../../models/model_{model_version}_{df.polarity}.keras'

# Define log probability
target_log_prob = utils.define_log_prob(model_path, data_path, specified_parameters, penalty=penalty, integrate=integrate, par_equals_perr=par_equals_perr)

Found 15 combinations to run MCMC on. Performing MCMC on index 0.


In [119]:
bins, bin_midpoints, observed, uncertainty = utils.load_data_ams(data_path, integrate)

print(f'Bins ({len(bins)}): {bins}')
print(f'Bin midpoints ({len(bins)}): {bin_midpoints}')
print(f'Observed: {observed}')

Bins (15): [0.28 0.76 0.83 0.9  0.98 1.06 1.16 1.26 1.38 1.51 1.66 1.83 2.02 2.23
 2.47]
Bin midpoints (15): [0.29933259 0.79422919 0.86429162 0.93914855 1.01921538 1.1088733
 1.2089665  1.31863566 1.44353732 1.58322456 1.74292857 1.92265442
 2.1224043  2.34693417]
Observed: [ 30.39873 381.5869  378.8598  423.6459  382.2285  427.3153  394.8742
 407.5119  394.1613  395.7958  413.7841  395.2794  356.6092  297.1244 ]


## Check that calculated chi2 matches results

In [120]:
results_dir = '../../../results/' + hmc_version + '/'

# Load results_dir/logprobs_{SLURM_ARRAY_TASK_ID}_yearly_{year}.csv, and find the absolute value max logprob value and it's index
logprobs = pd.read_csv(results_dir + f'logprobs_{SLURM_ARRAY_TASK_ID}_yearly_{year}_neg.csv', header=None)
max_logprob = logprobs.max()
max_logprob_idx = logprobs.idxmax()

print(f'Max logprob: {max_logprob[0]} at index {max_logprob_idx[0]}')

Max logprob: -1.5840998888015747 at index 71052


In [121]:
# Get samples from results_dir/samples_{SLURM_ARRAY_TASK_ID}_yearly_{year}_neg.csv
samples = pd.read_csv(results_dir + f'samples_{SLURM_ARRAY_TASK_ID}_yearly_{year}_neg.csv', header=None)

# Get the sample at the max logprob index
max_logprob_sample = samples.iloc[max_logprob_idx[0]]

print(f'Max logprob sample: {max_logprob_sample}')

Max logprob sample: 0    431.610474
1      1.533347
2      1.112190
3      0.408744
4      0.431167
Name: 71052, dtype: float64


In [122]:
xs = []
for i in range(5):
    xs.append(max_logprob_sample[i])

xs = np.array(xs)

print(f'xs: {xs}')

# Transform xs
xs = transform_input(xs)

print(f'Transformed xs: {xs}')

xs: [4.31610474e+02 1.53334725e+00 1.11219001e+00 4.08743829e-01
 4.31167215e-01]
Transformed xs: [0.43066295 0.87180558 0.54783847 0.00460202 0.0164038 ]


### Break apart define_log_prob function and check each piece

In [123]:
# Load trained NN model that maps 8 parameters to predicted flux at RIGIDITY_VALS.
model = keras.models.load_model(model_path)
model.run_eagerly = True # Settable attribute (in elegy). Required to be true for ppmodel.

# Load observation data from Claudio
if integrate:
    bins, r1r2, observed, uncertainty = utils.load_data_ams(data_path, integrate)
else:
    bins, bin_midpoints, observed, uncertainty = utils.load_data_ams(data_path, integrate)

# Transform input parameters to be in range 0--1.
parameters_specified_transformed = transform_input(jnp.array(specified_parameters))
print(f'specified_parameters: {specified_parameters}')
print(f'parameters_specified_transformed: {parameters_specified_transformed}')

specified_parameters: (68.69, 7.03, 538.4)
parameters_specified_transformed: [0.8081177 0.6471429 0.4613334]


In [124]:
# If par==perr, then only predicting ['cpa', 'pwr1par', 'pwr2par']. Need to create array of ['cpa', 'pwr1par', 'pwr1par', 'pwr2par', 'pwr2par']
if par_equals_perr:
    xs = jnp.array([xs[0], xs[1], xs[1], xs[2], xs[2]])

# Include logprior in loglikelihood. This keeps HMC from going off into no-mans land.
nlogprior = 0.
for i in range(5):
    nlogprior += penalty * jnp.abs((jnp.minimum(0., xs[i]))) # Penalty for being <0
    nlogprior += penalty * jnp.abs((jnp.maximum(1., xs[i]) - 1.))  # Penalty for being >1

# log_prob = -chi2/2.  - nlogprior, so using nlogprior find the chi2 value associated with the max logprob
chi2_max_logprob = -2. * (max_logprob[0] + nlogprior)
print(f'Max logprob Chi2: {chi2_max_logprob}')

Max logprob Chi2: 3.1681997776031494


In [125]:
# Batch parameters and predict with model
batch = utils._form_batch(xs, parameters_specified_transformed)
yhat = model(batch)    
yhat = yhat[0,:]  # Remove batch dimension.
yhat = yhat.numpy()

# Prepare yhat and rigidity for integration/interpolation
yhat = utils.untransform_output(yhat.reshape((1,-1))).reshape(-1) # Undo scaling and minmax.
log_yhat = jnp.log(yhat)
log_rigidity = jnp.log(jnp.array(RIGIDITY_VALS))

# Compute chi2
chi2 = CalculateChi2(log_rigidity, log_yhat)
yhat_interp_integrated = []

if integrate:
    # Compute integral for each r1-r2 range
    for x1, x2 in r1r2:
        integral = chi2.compute_integral(x1, x2) / (x2 - x1)
        yhat_interp_integrated.append(integral)
else:
    # Compute interpolated value at each bin_midpoint
    for x in bin_midpoints:
        interpolated = chi2.interpolate_model(x)
        yhat_interp_integrated.append(interpolated)

# Compute log prob
chi2 = (((jnp.asarray(yhat_interp_integrated) - observed)/uncertainty)**2)
cum_chi2 = jnp.sum(chi2)
log_prob = -cum_chi2/2.  - nlogprior

In [126]:
xs_untransformed = untransform_input(xs)
print(f"Max logprob model index (0-based): {max_logprob_idx[0]}; logprob = {max_logprob[0]:.3f}, chi2 = {chi2_max_logprob:.3f}; cpa = {xs_untransformed[0]:.2f}, pwr1par = {xs_untransformed[1]:.2f}, pwr1perr = {xs_untransformed[2]:.2f}, pwr2par = {xs_untransformed[3]:.2f}, pwr2perr = {xs_untransformed[4]:.2f}")
print(f"Calculated chi2 and logprob from sample parameters: logprob = {log_prob:.3f}; chi2 = {cum_chi2:.3f}\n")

for i in range(len(yhat_interp_integrated)):
    print(f'rig = {bin_midpoints[i]:.6f} [{bins[i]:.2f}, {bins[i+1]:.2f}]; data = {observed[i]:.6f}',
          f'; unc = {uncertainty[i]:.6f}; mod: {yhat_interp_integrated[i]:.6f}',
          f'; cum_chi2 = {(chi2[:i+1].sum()):.6f}')

Max logprob model index (0-based): 71052; logprob = -1.584, chi2 = 3.168; cpa = 431.61, pwr1par = 1.53, pwr1perr = 1.11, pwr2par = 0.41, pwr2perr = 0.43
Calculated chi2 and logprob from sample parameters: logprob = -1.584; chi2 = 3.168

rig = 0.299333 [0.28, 0.76]; data = 30.398730 ; unc = 0.352263; mod: 30.417137 ; cum_chi2 = 0.002731
rig = 0.794229 [0.76, 0.83]; data = 381.586900 ; unc = 76.412730; mod: 293.324677 ; cum_chi2 = 1.336923
rig = 0.864292 [0.83, 0.90]; data = 378.859800 ; unc = 75.866600; mod: 328.203674 ; cum_chi2 = 1.782746
rig = 0.939149 [0.90, 0.98]; data = 423.645900 ; unc = 84.835040; mod: 359.885223 ; cum_chi2 = 2.347625
rig = 1.019215 [0.98, 1.06]; data = 382.228500 ; unc = 76.541200; mod: 392.363617 ; cum_chi2 = 2.365159
rig = 1.108873 [1.06, 1.16]; data = 427.315300 ; unc = 85.569810; mod: 408.579590 ; cum_chi2 = 2.413099
rig = 1.208967 [1.16, 1.26]; data = 394.874200 ; unc = 79.073500; mod: 425.897736 ; cum_chi2 = 2.567028
rig = 1.318636 [1.26, 1.38]; data = 40

# Check older data

In [127]:
SLURM_ARRAY_TASK_ID = 0
SLURM_ARRAY_JOB_ID = 0
DEBUG = True

# Version specifications
model_version = 'v3.0' # v2.0 is MSE NN, v3.0 is MAE NN
hmc_version = 'v24.3'
file_version = '2023'

# Select experiment parameters
df = index_mcmc_runs(file_version=file_version)  # List of all experiments (0-209) for '2023', 0-14 for '2024'
print(f'Found {df.shape[0]} combinations to run MCMC on. Performing MCMC on index {SLURM_ARRAY_TASK_ID}.')
df = df.iloc[SLURM_ARRAY_TASK_ID]

# Load observation data and define logprob. 
if file_version == '2023': 
    data_path = f'../../data/oct2022/{df.experiment_name}/{df.experiment_name}_{df.interval}.dat'  # This data is the same.
elif file_version == '2024': 
    year = 2000 + SLURM_ARRAY_TASK_ID # assumes only negative intervals. If otherwise, fix this
    data_path = f'../../data/2024/yearly/{year}.dat'
else:
    raise ValueError(f"Invalid file_version {file_version}. Must be '2023' or '2024'.")

model_path = f'../../models/model_{model_version}_{df.polarity}.keras'

df.head()

Found 133 combinations to run MCMC on. Performing MCMC on index 0.


interval     20110520-20110610
alpha                    51.49
cmf                       4.85
vspoles                 632.52
alpha_std                10.69
Name: 0, dtype: object

In [129]:
bins, bin_midpoints, observed, uncertainty = utils.load_data_ams(data_path, integrate)
print(f"Loaded dataset from {data_path}.")
print(f'Bins: {bins}')
print(f'Observed: {observed}')
print(f'Uncertainty: {uncertainty}')

Loaded dataset from ../../data/oct2022/AMS02_H-PRL2021/AMS02_H-PRL2021_20110520-20110610.dat.
Bins: [  1.     1.16   1.33   1.51   1.71   1.92   2.15   2.4    2.67   2.97
   3.29   3.64   4.02   4.43   4.88   5.37   5.9    6.47   7.09   7.76
   8.48   9.26  10.1   11.    13.    16.6   22.8   33.5   48.5   69.7
 100.  ]
Observed: [9.542576e+02 9.411921e+02 8.769211e+02 8.003528e+02 7.088688e+02
 6.185521e+02 5.311145e+02 4.512981e+02 3.792883e+02 3.166651e+02
 2.633877e+02 2.184992e+02 1.800327e+02 1.470573e+02 1.199892e+02
 9.722954e+01 7.856188e+01 6.343624e+01 5.116549e+01 4.125726e+01
 3.316556e+01 2.672519e+01 2.151074e+01 1.559146e+01 9.113706e+00
 4.317174e+00 1.667450e+00 5.843280e-01 2.086225e-01 7.601720e-02]
Uncertainty: [2.811600e+01 2.148905e+01 1.643800e+01 1.293201e+01 1.028839e+01
 8.293686e+00 6.719937e+00 5.448942e+00 4.419263e+00 3.583056e+00
 2.932180e+00 2.403835e+00 1.970323e+00 1.589138e+00 1.290966e+00
 1.053403e+00 8.565820e-01 6.939785e-01 5.606656e-01 4.578981